In [1]:
from graph import *

### Problem 2

In [2]:
def load_map(mapFilename):
    """ 
    Parses the map file and constructs a directed graph

    Parameters: 
        mapFilename : name of the map file

    Assumes:
        Each entry in the map file consists of the following four positive 
        integers, separated by a blank space:
            From To TotalDistance DistanceOutdoors
        e.g.
            32 76 54 23
        This entry would become an edge from 32 to 76.

    Returns:
        a directed graph representing the map
    """
    # TODO
    print "Loading map from file..."
    mapFile = open(mapFilename, 'r')
    g = WeightedDigraph()
    for line in mapFile:
        line = line.split()
        node1 = Node(line[0])
        node2 = Node(line[1])
        if node1 not in g.nodes:
            g.addNode(node1)
        if node2 not in g.nodes:
            g.addNode(node2)
        edge = WeightedEdge(node1, node2, eval(line[2]), eval(line[3]))
        g.addEdge(edge)
    return g

In [3]:
mitMap = load_map("mit_map.txt")
print isinstance(mitMap, Digraph)

Loading map from file...
True


In [4]:
print isinstance(mitMap, WeightedDigraph)

True


In [5]:
print set(mitMap.nodes)#所有教学楼的楼号

set([36, 62, 64, 66, 68, 24, 26, 48, 46, 18, 1, 3, 2, 5, 4, 7, 6, 9, 8, 39, 76, 12, 10, 13, 38, 14, 16, 33, 54, 57, 56, 37, 50, 35, 34, 32, 31])


In [6]:
s = {}
for start in mitMap.edges:
    s[start] = []#存储以start节点作为起点的有权边
    for t in mitMap.edges[start]:
        s[start].append([t[0], (float(t[1]), float(t[2]))])
print s[Node("36")]#输出起点为36的有权边

[[32, (70.0, 0.0)], [26, (34.0, 0.0)], [34, (25.0, 0.0)], [46, (80.0, 40.0)], [48, (100.0, 80.0)]]


### Problem 3

求出满足约束的“最短路径”。已知带权重的有向图`mitMap`已被建立好，可用作程序测试。

In [7]:
mitMap

如果想从点1到点4：

In [8]:
paths = []

def getPath(graph, start, end, path = []):
    path = path + [start]
    if start == end:
        paths.append(path)
    for node in graph.childrenOf(start):
        if node not in path: 
            getPath(graph, node, end, path)

getPath(mitMap, Node('1'), Node('4'))

In [9]:
len(paths)

55179

存在`1->2->3->4`这一通路。

In [24]:
[path for path in paths if len(path) == 4]

[[1, 2, 10, 4], [1, 2, 3, 4], [1, 3, 10, 4], [1, 3, 2, 4]]

如果先找出所有连通路径（而不考虑距离约束），会得到55179个结果，如果对这么多结果进行遍历过滤，时间花费太大。有下面的实验可以看出，想从1走到4，存在非常多路径，而且不同路径之间差距很大。

In [10]:
lengthOfPaths = [len(path) for path in paths]

In [11]:
min(lengthOfPaths)

2

In [12]:
lengthOfPaths.index(2)

11275

In [13]:
print paths[11275]

[1, 4]


In [14]:
max(lengthOfPaths)

31

In [15]:
lengthOfPaths.index(31)

5434

In [16]:
print paths[5434]

[1, 2, 6, 8, 16, 56, 18, 54, 66, 68, 76, 32, 46, 48, 36, 26, 12, 24, 34, 38, 39, 13, 31, 37, 35, 33, 9, 7, 3, 10, 4]


In [17]:
for i in range(1,5):
    print "From %d to" % i, mitMap.edges[Node(i)]

From 1 to [(2, 75, 60), (4, 80, 65), (3, 36, 0), (5, 32, 0)]
From 2 to [(6, 41, 0), (14, 51, 0), (4, 36, 0), (10, 70, 50), (3, 70, 50), (1, 75, 60)]
From 3 to [(10, 32, 0), (4, 60, 50), (2, 70, 50), (1, 36, 0), (7, 25, 0)]
From 4 to [(12, 56, 0), (8, 47, 0), (2, 36, 0), (10, 47, 0), (3, 60, 50), (1, 80, 65)]


如果走`1->2->3->4`的话，室外距离是60+50+50=160。

考虑按照指示，先找出满足“室外距离”约束的路径，随后对这些路径进行“总距离”约束的过滤。

In [38]:
#取出graph中连接fromNode和toNode的有权边的两个权值：总距离和室外距离
#主程序的逻辑可以确保存在自fromNode到toNode的有权边

def getWeightOfEdge(graph, fromNode, toNode):
    weightedEdges = graph.edges[fromNode]
    theEdge = [edge for edge in weightedEdges if str(edge[0]) == toNode.getName()][0]
    return theEdge[1], theEdge[2]

print getWeightOfEdge(mitMap, Node("1"), Node("2"))
print getWeightOfEdge(mitMap, Node("2"), Node("3"))
print getWeightOfEdge(mitMap, Node("3"), Node("4"))

(75, 60)
(70, 50)
(60, 50)


在getPath的基础上，加入室外距离约束。

In [47]:
pathsSatisfyMaxOutdoors = []

def getPathSatisfyMaxOutdoors(graph, start, end, path, nowDistOutdoors, maxDistOutdoors):
    path = path + [start]
    if start == end:
        pathsSatisfyMaxOutdoors.append(path)
    for node in graph.childrenOf(start):
        if node not in path:
            newDist = getWeightOfEdge(graph, start, node)[1]
            if nowDistOutdoors + newDist <= maxDistOutdoors:
                getPathSatisfyMaxOutdoors(graph, node, end, path, nowDistOutdoors + newDist, maxDistOutdoors)

getPathSatisfyMaxOutdoors(mitMap, Node("1"), Node("4"), [], 0, 150)

把室外距离约束设为150，则把`[1,2,3,4]`筛除掉，说明算法正确。

In [48]:
[path for path in pathsSatisfyMaxOutdoors if len(path) == 4]

[[1, 2, 10, 4], [1, 3, 10, 4], [1, 3, 2, 4]]

对满足室外距离约束的路径进行基于总距离约束的过滤，同时找出其中满足双约束的最短路径。对于每一条满足室外距离约束的路径，看是否满足总距离的约束。若满足，检查是否是最短路径。

In [49]:
def getShortestPathSatisfyBothConstraints(graph, paths, maxTotalDist):
    minDist = 10000000
    theShortestPath = []
    for path in paths:
        nowTotalDist = 0
        exceedMaxTotalDist = False
        for i in range(len(path) - 1):
            curBldg = Node(str(path[i]))
            nxtBldg = Node(str(path[i + 1]))
            theDist = getWeightOfEdge(graph, curBldg, nxtBldg)[0]
            nowTotalDist += theDist
            if nowTotalDist > maxTotalDist:
                exceedMaxTotalDist = True
                break
        if exceedMaxTotalDist:
            continue
        if nowTotalDist < minDist:
            minDist = nowTotalDist
            theShortestPath = path
    return theShortestPath

得到室外距离小于150， 总距离小于200的最短路径。

In [50]:
getShortestPathSatisfyBothConstraints(mitMap, pathsSatisfyMaxOutdoors, 200)

[1, 4]

`Python`支持在函数内定义函数，因此可以用函数式编程的思想，把多个辅助函数组合起来。

In [61]:
def fact(k):
    def iterFact(n, result):
        if n > k:
            return result
        return iterFact(n + 1, n * result)
    return iterFact(1, 1)

In [62]:
fact(5)

120

下面用`bruteForceSearch`将上面的辅助函数组合起来。

In [68]:
def bruteForceSearch(digraph, start, end, maxTotalDist, maxDistOutdoors):
    """
    Finds the shortest path from start to end using brute-force approach.
    The total distance traveled on the path must not exceed maxTotalDist,
    and the distance spent outdoor on this path must not exceed
    maxDistOutdoors.

    Parameters:
        digraph: instance of class Digraph or its subclass
        start, end: start & end building numbers (strings)
        maxTotalDist : maximum total distance on a path (integer)
        maxDistOutdoors: maximum distance spent outdoors on a path (integer)

    Assumes:
        start and end are numbers for existing buildings in graph

    Returns:
        The shortest-path from start to end, represented by
        a list of building numbers (in strings), [n_1, n_2, ..., n_k],
        where there exists an edge from n_i to n_(i+1) in digraph,
        for all 1 <= i < k.

        If there exists no path that satisfies maxTotalDist and
        maxDistOutdoors constraints, then raises a ValueError.
    """
    
    #根据注释，输入的start和end是代表楼号的字符串。首先需要转换为节点对象
    start = Node(start)
    end = Node(end)
    
    #返回连接两点的有权边的权值
    def getWeightOfEdge(fromNode, toNode):
        weightedEdges = digraph.edges[fromNode]
        theEdge = [edge for edge in weightedEdges if str(edge[0]) == toNode.getName()][0]
        #在MITx平台上必须加eval
        #return eval(theEdge[1]), eval(theEdge[2])
        return theEdge[1], theEdge[2]
    
    #用于存放满足室外距离约束的所有路径
    paths = []
    
    #基于室外距离约束进行路径筛除
    def getPathSatisfyMaxOutdoors(cur, path, nowDistOutdoors):
        path = path + [cur]
        if cur == end:
            paths.append(path)
        for node in digraph.childrenOf(cur):
            if node not in path:
                newDist = getWeightOfEdge(cur, node)[1]
                if nowDistOutdoors + newDist <= maxDistOutdoors:
                    getPathSatisfyMaxOutdoors(node, path, nowDistOutdoors + newDist)
    
    getPathSatisfyMaxOutdoors(start, [], 0)
    if paths == []:
        raise ValueError
    
    #找出满足双重约束的最短路径
    def getShortestPathSatisfyBothConstraints():
        minDist = 10000000
        theShortestPath = []
        for path in paths:
            nowTotalDist = 0
            exceedMaxTotalDist = False
            for i in range(len(path) - 1):
                curBldg = Node(str(path[i]))
                nxtBldg = Node(str(path[i + 1]))
                theDist = getWeightOfEdge(curBldg, nxtBldg)[0]
                nowTotalDist += theDist
                if nowTotalDist > maxTotalDist:
                    exceedMaxTotalDist = True
                    break
            if exceedMaxTotalDist:
                continue
            if nowTotalDist < minDist:
                minDist = nowTotalDist
                theShortestPath = path
        return theShortestPath
    
    theShortestPath = getShortestPathSatisfyBothConstraints()
    if theShortestPath == []:
        raise ValueError
    
    theShortestPath = [str(bldgNo) for bldgNo in theShortestPath]
    return theShortestPath

In [69]:
bruteForceSearch(mitMap, "1", "4", 200, 150)

['1', '4']

In [70]:
bruteForceSearch(mitMap, "1", "5", 210, 100)

['1', '5']

In [71]:
bruteForceSearch(mitMap, "36", "32", 2000, 2000)

['36', '32']

In [72]:
getWeightOfEdge(mitMap, Node(36), Node(32))

(70, 0)

这个函数看上去很对，但在交作业的时候一直报一些莫名其妙的错误……让我再确认一下函数有没有写错。

In [73]:
bruteForceSearch(mitMap, "36", "1", 400, 400)

['36', '34', '24', '13', '10', '3', '1']

先计算出这一路径的室外距离，然后把约束降至该值以下，看是否返回一个别的路径。如果写对的话，肯定要出现别的路径。这一路径不如前一路径的总距离那么短，但是它的室外距离满足新规定。

In [74]:
def getWeightOfPath(graph, path):
    totalDist = 0
    distOutdoors = 0
    for i in range(len(path) - 1):
        fromNode = Node(path[i])
        toNode = Node(path[i + 1])
        d1, d2 = getWeightOfEdge(graph, fromNode, toNode)
        totalDist += d1
        distOutdoors += d2
    return totalDist, distOutdoors

In [75]:
getWeightOfPath(mitMap, [36, 34, 24, 13, 10, 3, 1])

(185, 30)

In [76]:
bruteForceSearch(mitMap, "36", "1", 400, 29)

['36', '26', '12', '4', '10', '3', '1']

In [77]:
getWeightOfPath(mitMap, [36, 26, 12, 4, 10, 3, 1])

(235, 25)

可以发现，新路径不如原路径那么短，但室外距离满足29这一约束。

In [78]:
bruteForceSearch(mitMap, "36", "1", 400, 24)

['36', '34', '24', '12', '4', '10', '3', '1']

In [79]:
getWeightOfPath(mitMap, [36, 34, 24, 12, 4, 10, 3, 1])

(256, 0)

甚至发现了一条室外距离为0的路径。下面要出现`ValueError`了。

In [80]:
bruteForceSearch(mitMap, "36", "1", 255, 1)

ValueError: 

最后发现，必须在辅助函数`getWeightOfEdge`的返回项中加上`eval()`才能通过测试。平台的解释器把该函数的返回结果认作`str`对象。

### Problem 4

希望在检索的时候记录最短路径，减少搜索深度。因此，搜索时需要同时记录总距离和室外距离，并依照总距离，剪枝。

In [139]:
def directedDFS(digraph, start, end, maxTotalDist, maxDistOutdoors):
    """
    Finds the shortest path from start to end using directed depth-first
    search approach. The total distance traveled on the path must not
    exceed maxTotalDist, and the distance spent outdoor on this path
    must not exceed maxDistOutdoors.

    Parameters:
        digraph: instance of class Digraph or its subclass
        start, end: start & end building numbers (strings)
        maxTotalDist : maximum total distance on a path (integer)
        maxDistOutdoors: maximum distance spent outdoors on a path (integer)

    Assumes:
        start and end are numbers for existing buildings in graph

    Returns:
        The shortest-path from start to end, represented by
        a list of building numbers (in strings), [n_1, n_2, ..., n_k],
        where there exists an edge from n_i to n_(i+1) in digraph,
        for all 1 <= i < k.

        If there exists no path that satisfies maxTotalDist and
        maxDistOutdoors constraints, then raises a ValueError.
    """
    
    start = Node(start)
    end = Node(end)
    
    #返回连接两点的有权边的权值
    def getWeightOfEdge(fromNode, toNode):
        weightedEdges = digraph.edges[fromNode]
        theEdge = [edge for edge in weightedEdges if str(edge[0]) == toNode.getName()][0]
        #在MITx平台上，要加上eval()
        return theEdge[1], theEdge[2]
    
    #返回路径的距离
    def getPathDist(path):
        if path == None:
            return maxTotalDist, 0
        sumTotalDist = 0
        sumOutdoorsDist = 0
        for i in range(len(path) - 1):
            cur = Node(path[i])
            nxt = Node(path[i + 1])
            totalDist, outdoorsDist = getWeightOfEdge(cur, nxt)
            sumTotalDist += totalDist
            sumOutdoorsDist += outdoorsDist
        return sumTotalDist, sumOutdoorsDist
    
    def search(cur, path, nowTD, nowOD, shortest = None):
        
        #给总距离设一个上界
        #如果还没找到一条通路，则总距离上界为maxTotalDist
        #如果已经找到了通路，则总距离上界为当前最短通路的总距离
        minDist = getPathDist(shortest)[0]
        
        path = path + [cur]
        if cur == end:
            return path
        for node in digraph.childrenOf(cur):
            if node not in path:
                newTD, newOD = getWeightOfEdge(cur, node)
                cond1 = (nowTD + newTD < minDist)
                cond2 = (nowOD + newOD <= maxDistOutdoors)
                if cond1 and cond2:
                    newPath = search(node, path, nowTD+newTD, nowOD+newOD, shortest)
                    if newPath != None:
                        shortest = newPath
        return shortest
    
    theShortestPath = search(start, [], 0, 0)
    
    if theShortestPath == None:
        raise ValueError
    
    theShortestPath = [str(bldgNo) for bldgNo in theShortestPath]
    return theShortestPath

速度实在是十分地块。

In [140]:
directedDFS(mitMap, "36", "1", 400, 400)

['36', '34', '24', '13', '10', '3', '1']

In [136]:
getWeightOfPath(mitMap, ['36', '34', '24', '13', '10', '3', '1'])

(185, 30)

In [141]:
directedDFS(mitMap, "36", "1", 255, 1)

ValueError: 

最后还是有数个测试案例没通过。按照指示，使用`ps5.py`中的测试样例。只测试`directedDFS`。

In [145]:
# Uncomment below when ready to test
#### NOTE! These tests may take a few minutes to run!! ####
if __name__ == '__main__':
#     Test cases
    mitMap = load_map("mit_map.txt")
    print isinstance(mitMap, Digraph)
    print isinstance(mitMap, WeightedDigraph)
    print 'nodes', mitMap.nodes
    print 'edges', mitMap.edges


    LARGE_DIST = 1000000

#     Test case 1
    print "---------------"
    print "Test case 1:"
    print "Find the shortest-path from Building 32 to 56"
    expectedPath1 = ['32', '56']
    #brutePath1 = bruteForceSearch(mitMap, '32', '56', LARGE_DIST, LARGE_DIST)
    dfsPath1 = directedDFS(mitMap, '32', '56', LARGE_DIST, LARGE_DIST)
    print "Expected: ", expectedPath1
    print "Brute-force: ", brutePath1
    print "DFS: ", dfsPath1
    print "Correct? BFS: {0}; DFS: {1}".format(expectedPath1 == brutePath1, expectedPath1 == dfsPath1)

Loading map from file...
True
True
nodes set([54, 50, 62, 64, 66, 68, 24, 26, 48, 46, 1, 3, 2, 5, 4, 7, 6, 9, 8, 13, 76, 38, 10, 39, 12, 14, 16, 33, 32, 57, 56, 37, 36, 35, 34, 18, 31])
edges {54: [(56, 40, 30), (66, 45, 35), (18, 20, 10), (62, 20, 10), (14, 70, 60), (50, 80, 70)], 50: [(14, 50, 23), (14, 25, 20)], 62: [(54, 20, 10), (64, 30, 20)], 64: [(62, 30, 20)], 66: [(68, 51, 0), (56, 40, 0), (76, 130, 100), (32, 70, 60)], 68: [(32, 110, 80), (76, 72, 30), (66, 51, 0), (56, 80, 70)], 24: [(13, 35, 30), (26, 25, 20), (34, 27, 0), (12, 33, 0)], 26: [(36, 34, 0), (16, 45, 0), (12, 30, 25), (24, 25, 20)], 48: [(32, 80, 50), (36, 100, 80), (46, 25, 10)], 46: [(32, 90, 40), (36, 80, 40), (48, 25, 10)], 1: [(2, 75, 60), (4, 80, 65), (3, 36, 0), (5, 32, 0)], 3: [(10, 32, 0), (4, 60, 50), (2, 70, 50), (1, 36, 0), (7, 25, 0)], 2: [(6, 41, 0), (14, 51, 0), (4, 36, 0), (10, 70, 50), (3, 70, 50), (1, 75, 60)], 5: [(1, 32, 0), (7, 20, 0)], 4: [(12, 56, 0), (8, 47, 0), (2, 36, 0), (10, 47, 0), 

In [149]:
#     Test case 2
print "---------------"
print "Test case 2:"
print "Find the shortest-path from Building 32 to 56 without going outdoors"
expectedPath2 = ['32', '36', '26', '16', '56']
#brutePath2 = bruteForceSearch(mitMap, '32', '56', LARGE_DIST, 0)
dfsPath2 = directedDFS(mitMap, '32', '56', LARGE_DIST, 0)
print "Expected: ", expectedPath2
#print "Brute-force: ", brutePath2
print "DFS: ", dfsPath2
print "Correct? BFS: {0}; DFS: {1}".format(expectedPath2 == brutePath2, expectedPath2 == dfsPath2)

 ---------------
Test case 2:
Find the shortest-path from Building 32 to 56 without going outdoors
Expected:  ['32', '36', '26', '16', '56']
DFS:  ['32', '36', '26', '16', '56']


NameError: name 'brutePath2' is not defined

In [150]:
#     Test case 3
print "---------------"
print "Test case 3:"
print "Find the shortest-path from Building 2 to 9"
expectedPath3 = ['2', '3', '7', '9']
#brutePath3 = bruteForceSearch(mitMap, '2', '9', LARGE_DIST, LARGE_DIST)
dfsPath3 = directedDFS(mitMap, '2', '9', LARGE_DIST, LARGE_DIST)
print "Expected: ", expectedPath3
#print "Brute-force: ", brutePath3
print "DFS: ", dfsPath3
print "Correct? BFS: {0}; DFS: {1}".format(expectedPath3 == brutePath3, expectedPath3 == dfsPath3)

---------------
Test case 3:
Find the shortest-path from Building 2 to 9
Expected:  ['2', '3', '7', '9']
DFS:  ['2', '3', '7', '9']


NameError: name 'brutePath3' is not defined

In [151]:
#     print "---------------"
print "Test case 4:"
print "Find the shortest-path from Building 2 to 9 without going outdoors"
expectedPath4 = ['2', '4', '10', '13', '9']
#brutePath4 = bruteForceSearch(mitMap, '2', '9', LARGE_DIST, 0)
dfsPath4 = directedDFS(mitMap, '2', '9', LARGE_DIST, 0)
print "Expected: ", expectedPath4
#print "Brute-force: ", brutePath4
print "DFS: ", dfsPath4
print "Correct? BFS: {0}; DFS: {1}".format(expectedPath4 == brutePath4, expectedPath4 == dfsPath4)

Test case 4:
Find the shortest-path from Building 2 to 9 without going outdoors
Expected:  ['2', '4', '10', '13', '9']
DFS:  ['2', '4', '10', '13', '9']


NameError: name 'brutePath4' is not defined

In [152]:
#     Test case 5
print "---------------"
print "Test case 5:"
print "Find the shortest-path from Building 1 to 32"
expectedPath5 = ['1', '4', '12', '32']
#brutePath5 = bruteForceSearch(mitMap, '1', '32', LARGE_DIST, LARGE_DIST)
dfsPath5 = directedDFS(mitMap, '1', '32', LARGE_DIST, LARGE_DIST)
print "Expected: ", expectedPath5
#print "Brute-force: ", brutePath5
print "DFS: ", dfsPath5
print "Correct? BFS: {0}; DFS: {1}".format(expectedPath5 == brutePath5, expectedPath5 == dfsPath5)

---------------
Test case 5:
Find the shortest-path from Building 1 to 32
Expected:  ['1', '4', '12', '32']
DFS:  ['1', '4', '12', '32']


NameError: name 'brutePath5' is not defined

In [153]:
#     Test case 6
print "---------------"
print "Test case 6:"
print "Find the shortest-path from Building 1 to 32 without going outdoors"
expectedPath6 = ['1', '3', '10', '4', '12', '24', '34', '36', '32']
#brutePath6 = bruteForceSearch(mitMap, '1', '32', LARGE_DIST, 0)
dfsPath6 = directedDFS(mitMap, '1', '32', LARGE_DIST, 0)
print "Expected: ", expectedPath6
#print "Brute-force: ", brutePath6
print "DFS: ", dfsPath6
print "Correct? BFS: {0}; DFS: {1}".format(expectedPath6 == brutePath6, expectedPath6 == dfsPath6)

---------------
Test case 6:
Find the shortest-path from Building 1 to 32 without going outdoors
Expected:  ['1', '3', '10', '4', '12', '24', '34', '36', '32']
DFS:  ['1', '3', '10', '4', '12', '24', '34', '36', '32']


NameError: name 'brutePath6' is not defined

In [154]:
#     Test case 7
print "---------------"
print "Test case 7:"
print "Find the shortest-path from Building 8 to 50 without going outdoors"
bruteRaisedErr = 'No'
dfsRaisedErr = 'No'
try:
    bruteForceSearch(mitMap, '8', '50', LARGE_DIST, 0)
except ValueError:
    bruteRaisedErr = 'Yes'

try:
    directedDFS(mitMap, '8', '50', LARGE_DIST, 0)
except ValueError:
    dfsRaisedErr = 'Yes'

print "Expected: No such path! Should throw a value error."
print "Did brute force search raise an error?", bruteRaisedErr
print "Did DFS search raise an error?", dfsRaisedErr

---------------
Test case 7:
Find the shortest-path from Building 8 to 50 without going outdoors
Expected: No such path! Should throw a value error.
Did brute force search raise an error? Yes
Did DFS search raise an error? Yes


In [155]:
#     Test case 8
print "---------------"
print "Test case 8:"
print "Find the shortest-path from Building 10 to 32 without walking"
print "more than 100 meters in total"
bruteRaisedErr = 'No'
dfsRaisedErr = 'No'
try:
    bruteForceSearch(mitMap, '10', '32', 100, LARGE_DIST)
except ValueError:
    bruteRaisedErr = 'Yes'

try:
    directedDFS(mitMap, '10', '32', 100, LARGE_DIST)
except ValueError:
    dfsRaisedErr = 'Yes'

print "Expected: No such path! Should throw a value error."
print "Did brute force search raise an error?", bruteRaisedErr
print "Did DFS search raise an error?", dfsRaisedErr

---------------
Test case 8:
Find the shortest-path from Building 10 to 32 without walking
more than 100 meters in total
Expected: No such path! Should throw a value error.
Did brute force search raise an error? Yes
Did DFS search raise an error? Yes


明明这八个测试样例都通过了好吧……算了……